In [4]:
from pymongo import MongoClient
from datetime import datetime

def insert_data():
    client = MongoClient('mongodb://localhost:27017/')
    db = client.local  # 'local' 데이터베이스 사용

    db.books.delete_many({}) # 자꾸 결과가 중복되서 나와서 인공지능의 도움(Feat.GPT)

    # 책 데이터 삽입
    books = [
        {"title": "Kafka on the Shore", "author": "Haruki Murakami", "year": 2002},
        {"title": "Norwegian Wood", "author": "Haruki Murakami", "year": 1987},
        {"title": "1Q84", "author": "Haruki Murakami", "year": 2009}
    ]
    db.books.insert_many(books)

    # 영화 데이터 삽입
    movies = [
        {"title": "Inception", "director": "Christopher Nolan", "year": 2010, "rating": 8.8},
        {"title": "Interstellar", "director": "Christopher Nolan", "year": 2014, "rating": 8.6},
        {"title": "The Dark Knight", "director": "Christopher Nolan", "year": 2008, "rating": 9.0}
    ]
    db.movies.insert_many(movies)

    # 사용자 행동 데이터 삽입
    user_actions = [
        {"user_id": 1, "action": "click", "timestamp": datetime(2023, 4, 12, 8, 0)},
        {"user_id": 1, "action": "view", "timestamp": datetime(2023, 4, 12, 9, 0)},
        {"user_id": 2, "action": "purchase", "timestamp": datetime(2023, 4, 12, 10, 0)},
    ]
    db.user_actions.insert_many(user_actions)

    print("Data inserted successfully")
    client.close()

if __name__ == "__main__":
    insert_data()


 

Data inserted successfully


In [5]:
from pymongo import MongoClient

client = MongoClient('mongodb://localhost:27017/')
db = client.local  

# 장르 추가 #Feat.GPT
db.books.update_many(
    {},  
    {"$set": {"genre": "fantasy"}}  
)

client.close()

In [6]:
from pymongo import MongoClient

client = MongoClient('mongodb://localhost:27017/')
db = client.local 

def find_books_by_genre(db, genre):
    query = {"genre": genre}
    projection = {"_id": 0, "title": 1, "author": 1}
    books = db.books.find(query, projection)
    for book in books:
        print(book)

find_books_by_genre(db, 'fantasy')

{'title': 'Kafka on the Shore', 'author': 'Haruki Murakami'}
{'title': 'Norwegian Wood', 'author': 'Haruki Murakami'}
{'title': '1Q84', 'author': 'Haruki Murakami'}


In [7]:
def calculate_average_ratings(db):
    movies_collection = db.movies
    pipeline = [
        {"$group": {"_id": "$director", "average_rating": {"$avg": "$rating"}}},
        {"$sort": {"average_rating": -1}}
    ]

    results = movies_collection.aggregate(pipeline)
    for result in results:
        print(result)

calculate_average_ratings(db)

{'_id': 'Christopher Nolan', 'average_rating': 8.8}


In [8]:
def find_recent_actions_by_user(db, user_id, limit=5):
    user_actions_collection = db.user_actions
    query = {"user_id": user_id}
    sort_criteria = [("timestamp", -1)]

    actions = user_actions_collection.find(query).sort(sort_criteria).limit(limit)
    for action in actions:
        print(action)

find_recent_actions_by_user(db, 1)

{'_id': ObjectId('682190d130ea3a4796773b8a'), 'user_id': 1, 'action': 'view', 'timestamp': datetime.datetime(2023, 4, 12, 9, 0)}
{'_id': ObjectId('6821918eaebd84087f5353ef'), 'user_id': 1, 'action': 'view', 'timestamp': datetime.datetime(2023, 4, 12, 9, 0)}
{'_id': ObjectId('68219182aebd84087f5353e5'), 'user_id': 1, 'action': 'view', 'timestamp': datetime.datetime(2023, 4, 12, 9, 0)}
{'_id': ObjectId('68218ec430ea3a4796773b6c'), 'user_id': 1, 'action': 'seen', 'timestamp': datetime.datetime(2023, 4, 12, 9, 0)}
{'_id': ObjectId('6821906430ea3a4796773b7b'), 'user_id': 1, 'action': 'view', 'timestamp': datetime.datetime(2023, 4, 12, 9, 0)}


In [9]:
def count_books_by_year(db):
    books_collection = db.books
    pipeline = [
        {"$group": {"_id": "$year", "count": {"$sum": 1}}},
        {"$sort": {"count": -1}}
    ]

    results = books_collection.aggregate(pipeline)
    for result in results:
        print(result)

count_books_by_year(db)

{'_id': 2002, 'count': 1}
{'_id': 1987, 'count': 1}
{'_id': 2009, 'count': 1}


In [10]:
from datetime import datetime

def update_user_actions_before_date(db, user_id, date, old_action, new_action):
    user_actions_collection = db.user_actions
    query = {"user_id": user_id, "action": old_action, "timestamp": {"$lt": date}}
    update = {"$set": {"action": new_action}}

    result = user_actions_collection.update_many(query, update)
    print(f"Updated {result.modified_count} documents.")

update_user_actions_before_date(db, 1, datetime(2023, 4, 10), "view", "seen")

Updated 0 documents.
